In [1]:
from icecap import Project
import icecap as ice
import polars as pl 

In [2]:
proj = Project() 

proj['raw'] = proj.scan_csv("world_bank_gdp_data.csv")

input = proj['raw'].select(
    pl.col("Country Code").alias("Country"),
    pl.col("^[0-9]+.*$").cast(pl.Float32, strict=False).map_alias(lambda s: s[:4]))

input = input.melt(id_vars=["Country"], variable_name="Date", value_name="GDP") #, "pivot tall"

input = input.select(
    pl.col("Country"),
    pl.col("Date").cast(pl.Int32),
    pl.col("*").exclude("Country", "Date")
) #, "cast dates to number"

input = input.drop_nulls().sort(by=["Country", "Date"]) #, "drop nulls and sort"

proj['input'] = input


In [5]:
widget = proj['input'].inspect(adjust_lf=lambda x: x.head(5).collect())
widget

In [4]:
ice.traverse(proj['input'], named_only=True)

[<icecap.Frame at 0x10d822560>, <icecap.Frame at 0x10d823460>]